<a href="https://colab.research.google.com/github/zionsoumik/deception/blob/main/liar_bert%2Blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data1=pd.read_csv('train_liar.csv')
data2=pd.read_csv('val_liar.csv')
print(data1.head())

FileNotFoundError: ignored

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
!pip install torch==1.10.0
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


In [ ]:
!pip install transformers==4.12.5
import random
import time
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import StratifiedKFold
import datetime
# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import transformers as ppb
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
trainX=data1.iloc[:,1]
Ytrain=data1.iloc[:,0]
testX=data2.iloc[:,1]
Ytest=data2.iloc[:,0]

In [ ]:
max_len = 0
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
# For every sentence...
for sent in trainX:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

In [ ]:
train_input_ids = []
train_attention_masks = []
test_input_ids = []
test_attention_masks = []

# For every sentence...
for sent in trainX:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    train_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    train_attention_masks.append(encoded_dict['attention_mask'])

for sent in testX:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(Ytrain)


test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
test_labels = torch.tensor(Ytest)

# Print sentence 0, now as a list of IDs.
#print('Original: ', X[0])
#print('Token IDs:', input_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split
import torch.nn as nn
from torch.autograd import Variable
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = len(train_dataset)
val_size = len(val_dataset)
#kfold = KFold(n_splits=10, shuffle=True)
#train_indices, val_indices = train_test_split(list(range(len(labels))), test_size=0.2, stratify=labels, random_state=1) #random_state=1 epoch 6
# Divide the dataset by randomly selecting samples.
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
#train_dataset = torch.utils.data.Subset(dataset, train_indices)
#val_dataset = torch.utils.data.Subset(dataset, val_indices)
# Divide the dataset by randomly selecting samples.


print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

In [ ]:
class LSTM(nn.Module):

  def __init__(self):
        # python requires to call ancestor's initilizer manually!
        super(LSTM, self).__init__()
        self.num_layers = 1
        self.hidden_size = 200
        self.lstm = nn.LSTM(768, self.hidden_size//2, self.num_layers, batch_first=True, bidirectional=True).cuda()
        self.linear = nn.Linear(200, 6).cuda()

  def init_state(self, batch_size):
        #return (torch.zeros(self.num_layers, batch_size, self.hidden_size).cuda(),
                #torch.zeros(self.num_layers, batch_size, self.hidden_size).cuda())
         return (torch.randn(2, 1, self.hidden_size//2).cuda(),
                torch.randn(2, 1, self.hidden_size//2).cuda())       

  def forward(self, x):
        x = x.unsqueeze(0)
        #print(x.size())
        hs = self.init_state(1)
        x, hs = self.lstm(x, hs)
        x = x.reshape(-1, 200)
        x = self.linear(x).cuda()
        return x

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 6, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
from torch import nn

from transformers import BertForSequenceClassification
from torch.nn import CrossEntropyLoss, MSELoss


class BertConcatFeatures(BertForSequenceClassification,nn.Module):
    """
    A model for classification or regression which combines text, categorical,
    and numerical features. The text features are processed with BERT. All
    features are concatenated into a single vector, which is fed into an MLP
    for final classification / regression.

    This class expects a transformers.BertConfig object, and the config object
    needs to have three additional properties manually added to it:
      `text_feat_dim` - The length of the BERT vector.
      `cat_feat_dim` - The number of categorical features.
      `numerical_feat_dim` - The number of numerical features.
    """

    def __init__(self, config):
        
        # ====================
        #     BERT Setup
        # ====================

        # Call the constructor for the huggingface `BertForSequenceClassification` 
        # class, which will do all of the BERT-related setup. The resulting BERT
        # model is stored in `self.bert`. 
        super().__init__(config)
        
        # ==================================
        #     Feature Combination Setup
        # ==================================

        # Store the number of labels, which tells us whether this is a 
        # classification or regression task.
        #self.num_labels = config.num_labels

        # Calculate the combined vector length. 
        #combined_feat_dim = config.text_feat_dim + \
                                #config.numerical_feat_dim

        # Create a batch normalizer for the numerical features.
        #self.num_bn = nn.BatchNorm1d(config.numerical_feat_dim)

        # ====================
        #     MLP Setup
        # ====================

        # To setup the MLP, we need to specify the number of layers and the
        # number of neurons in each layer. The MultiModal-Toolkit has a formula
        # for picking these dimensions. Each layer of the MLP has 1/4th the 
        # number of neurons as the previous one. 
        
        # Dimensions of each MLP layer.
        #dims = []

        # Starting with the combined feature vector length...
        #dim = combined_feat_dim

        # Keep dividing by 4 until we drop below the number of outputs the MLP
        # needs to have. 
        #while True:
            
            # Divide by 4 and truncate to an integer.
            #im = dim // 4
            
            # If the resulting layer size would be smaller than the number of 
            # outputs, then we're done.
            #if dim <= self.num_labels:
                #break
            
            # Otherwise, store this as the next layer size.
            #dims.append(int(dim))

        # Print out the resulting MLP.                
        #print('MLP layer sizes:')
        #print('  Input:', combined_feat_dim)
        #print('  Hidden:', dims)
        #print('  Output:', self.num_labels)
        #print('')

        # Construct the MLP, specifying the number of inputs, outputs, and the
        # layer sizes.
        '''
        self.mlp = MLP(combined_feat_dim,
                        self.num_labels,
                        num_hidden_lyr=len(dims),
                        dropout_prob=0.1,
                        hidden_channels=dims,
                        bn=True)'''
        self.lstm = LSTM()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        class_weights=None,
        output_attentions=None,
        output_hidden_states=None
    ):
        '''
        Perform a forward pass of our model.
        
        This has the same inputs as `forward` in `BertForSequenceClassification`,
        but with two extra parameters:
          `cat_feats` - Tensor of categorical features.
          `numerical_feats` - Tensor of numerical features.  
        '''
        
        # ====================
        #        BERT
        # ====================
       
        # Run the text through the BERT model. Invoking `self.bert` returns 
        # outputs from the encoding layers, and not from the final classifier.
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        
        # outputs[0] - All of the output embeddings from BERT
        # outputs[1] - The [CLS] token embedding, with some additional "pooling"
        #              done.
        cls = outputs[1]
        
        # Apply dropout to the CLS embedding.
        cls = self.dropout(cls)

        # ==========================
        #    Concatenate Features
        # ==========================        

        # Apply batch normalization to the numerical features.        
        #numerical_feats = self.num_bn(numerical_feats)
    
        # Object sizes:
        #             cls   [batch size  x   768]
        # numerical_feats   [batch size  x   # numerical features]
        #       cat_feats   [batch size  x   # categorical features]
    
        # Simply concatenate everything into one vector.
        # For example, if we have 4 categ. and 3 numer. features, then the
        # result has 768 + 4 + 3 = 775 features.
        #combined_feats = torch.cat((cls, numerical_feats),dim=1)
        
        # ====================================
        #    Output Classifier / Regression
        # ====================================        
        
        # Run the the samples through the MLP.
        
        
        # Put the results into a Dictionary to return.
        
  
        #lstm_output, (h,c) = self.lstm(cls.v) ## extract the 1st token's embeddings
        #hidden = torch.cat((lstm_output[:,-1, :256],lstm_output[:,0, 256:]),dim=1)
        logits = self.lstm(cls) ### assuming that you are only using the output of the last LSTM cell to perform classification
        
        

        # TODO - Not sure what's going on with the outputs...

        if type(logits) is tuple:
            logits, classifier_layer_outputs = logits[0], logits[1]

        else:  # simple classifier
            classifier_layer_outputs = [cls, logits]
        
        
        # =================
        #       Loss
        # =================

        # Calculate loss, but only if labels were provided.
        # (Labels aren't provided at test time).
        
        if labels is not None:

            # Regression
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()

                labels = labels.float()

                loss = loss_fct(logits.view(-1), labels.view(-1))
            
            # Classification
            else:
                loss_fct = CrossEntropyLoss(weight=class_weights)
                
                labels = labels.long()
                
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        
        # If no labels are provided, set the loss to 'None'.
        else:
            loss = None
        
        results = {'loss': loss, 
                   'logits': logits,
                   'classifier_layer_outputs': classifier_layer_outputs}

        return results
        



In [ ]:
from transformers import BertConfig

# We'll need to a use a "BertConfig" object from the transformers library to
# specify our parameters. 

# First, specify the ordinary BERT parameters by taking them from the 
# 'bert-base-uncased' model. 
# Also set the number of labels.
config = BertConfig.from_pretrained(
        'bert-base-uncased',
        num_labels=6, # The number of output labels--1 for regression.
    )

# To set up the MLP, we need to know the combined vector length that will be
# sent into it. 

# Pass in the number of numerical and categorical features.
#config.numerical_feat_dim = numerical_feats.size()[1]
#config.cat_feat_dim = categorical_feats.size()[1]

# Pass in the size of the text embedding.
# The text feature dimension is the "hidden_size" parameter which 
# comes from BertConfig. The length is 768 in BERT-base (and most other BERT
# models).
#config.text_feat_dim = config.hidden_size # 768

# Now we're ready to do the actual set up of our model! Note that we're passing
# in the config object here.
model = BertConcatFeatures.from_pretrained(
    "bert-base-uncased",
    config = config
)

# Tell pytorch to run this model on the GPU.
desc = model.cuda()

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0
    
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 100 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        b_input_ids = batch[0].to(device)
        #b_numer_feats = batch[1].to(device) 
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        #print(b_labels)
        # Clear prior gradients. 
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. 
        # This will return the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels)

        loss = result['loss']
        logits = result['logits']

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    # Include the batch size so that we are looking at the average per-sample
    # loss.
    avg_train_loss = total_train_loss / (len(train_dataloader) * batch_size)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_loss = 0
    nb_eval_steps = 0
    total_eval_accuracy = 0
    
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        #b_numer_feats = batch[1].to(device) 
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result['loss']
        logits = result['logits']
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy() 
        total_eval_accuracy += flat_accuracy(logits, label_ids)       

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / (len(validation_dataloader) * batch_size)
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Valid. Accur.': avg_val_accuracy,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))